# Day16
Huggingface Datasets & Trainer API 进阶（自定义数据集、训练流程）    
本章涉及微调100M的模型，建议使用GPU。

## data preprocessing

GLUE（General Language Understanding Evaluation）基准测试包含 9 个数据集，每个数据集对应一个特定的自然语言理解任务。这些任务可以分为三类：
- 单句任务（Single-Sentence Tasks）：
    - CoLA（Corpus of Linguistic Acceptability）：判断句子是否语法正确。
    - SST-2（Stanford Sentiment Treebank）：情感分析，判断电影评论的情感是正面还是负面。
- 相似性和释义任务（Similarity and Paraphrase Tasks）：
    - MRPC（Microsoft Research Paraphrase Corpus）：判断两个句子是否为释义（语义等价）。
    - STS-B（Semantic Textual Similarity Benchmark）：评估两个句子的语义相似度（打分任务）。
    - QQP（Quora Question Pairs）：判断 Quora 上的两个问题是否语义等价。
- 自然语言推理任务（Natural Language Inference Tasks）：
    - MNLI（Multi-Genre Natural Language Inference）：判断一个假设是否可以从前提中推断出来（包含匹配和不匹配两个测试集）。
    - QNLI（Question-answering Natural Language Inference）：从 SQuAD 数据集改编，判断一个句子是否包含问题的答案。
    - RTE（Recognizing Textual Entailment）：判断一个句子是否可以从另一个句子中推断出来。
    - WNLI（Winograd Natural Language Inference）：从 Winograd Schema Challenge 改编，解决代词指代问题。

In [10]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding, TrainingArguments
import numpy as np
import evaluate

In [ ]:
raw_datasets = load_dataset("glue", "mrpc")
raw_datasets

README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/649k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/75.7k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/308k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

1. **加载数据集**：
    - 使用 load_dataset() 从 Hub 或本地路径加载数据集。
    - 数据集通常分为 train、validation 和 test 三个拆分（split）。
2. **访问数据**：
    - 数据以类似字典的形式存储，支持切片、迭代和列访问。
    - 每个样本是一个字典，键对应特征（如 text、label）。
3. **处理数据**：
    - 支持映射（map）、过滤（filter）、分片（shard）等操作。
    - 可以无缝与 transformers 的 tokenizer 结合。
4. **输出**：
    - 数据可以直接访问，或转换为 PyTorch/TensorFlow 张量。

代码示例

In [ ]:
# 1. 加载数据集
dataset = load_dataset(
    "emotion",                  # 数据集名称
    split="train",              # 指定拆分（可选：train/validation/test）
    download_mode="reuse_dataset_if_exists",  # 复用已下载的数据
)

# 2. 查看数据集基本信息
print("数据集信息:", dataset)
print("特征:", dataset.features)
print("样本数量:", len(dataset))
print("第一个样本:", dataset[0])

# 3. 访问和切片数据
sample_text = dataset["text"][:3]  # 前 3 个样本的文本
sample_labels = dataset["label"][:3]  # 前 3 个样本的标签
print("\n前 3 个样本文本:", sample_text)
print("前 3 个样本标签:", sample_labels)

# 4. 结合 tokenizer 预处理
# 该函数接收一个字典（与 dataset 的项类似）并返回一个包含 id(input_ids) ，(attention_mask) 和 token_type_ids 键的新字典
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )

# 应用映射函数
# Datasets 库中的数据集是以 Apache Arrow 格式存储在磁盘上的，因此你只需将接下来要用的数据加载在内存中，而不是加载整个数据集
tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,              # 批量处理
    remove_columns=["text"],   # 删除原始文本列
    desc="Tokenizing dataset"  # 进度条描述
)
print("\n预处理后的第一个样本:", tokenized_dataset[0])

# 5. 过滤数据（示例：只保留标签为 0 的样本）
filtered_dataset = dataset.filter(
    lambda example: example["label"] == 0,
    desc="Filtering for label 0"
)
print("\n过滤后样本数量:", len(filtered_dataset))
print("过滤后第一个样本:", filtered_dataset[0])

# 6. 转换为 PyTorch 格式
tokenized_dataset.set_format(
    type="torch",              # 转换为 PyTorch 张量
    columns=["input_ids", "attention_mask", "label"]
)
print("\n转换为 PyTorch 后的第一个样本:", tokenized_dataset[0])

# 7. 批量访问（示例：取前 2 个样本）
batch = tokenized_dataset[:2]
print("\n批量数据 (input_ids):", batch["input_ids"].shape)
print("批量数据 (labels):", batch["label"])

README.md:   0%|          | 0.00/9.05k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/127k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/129k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

...

数据集信息: Dataset({
    features: ['text', 'label'],
    num_rows: 16000
})
特征: {'text': Value(dtype='string', id=None), 'label': ClassLabel(names=['sadness', 'joy', 'love', 'anger', 'fear', 'surprise'], id=None)}
样本数量: 16000
第一个样本: {'text': 'i didnt feel humiliated', 'label': 0}

前 3 个样本文本: ['i didnt feel humiliated', 'i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake', 'im grabbing a minute to post i feel greedy wrong']
前 3 个样本标签: [0, 0, 3]


Tokenizing dataset:   0%|          | 0/16000 [00:00<?, ? examples/s]


预处理后的第一个样本: {'label': 0, 'input_ids': [101, 1045, 2134, 2102, 2514, 26608, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

Filtering for label 0:   0%|          | 0/16000 [00:00<?, ? examples/s]


过滤后样本数量: 4666
过滤后第一个样本: {'text': 'i didnt feel humiliated', 'label': 0}

转换为 PyTorch 后的第一个样本: {'label': tensor(0), 'input_ids': tensor([  101,  1045,  2134,  2102,  2514, 26608,   102,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0

**输入参数**

1. **load_dataset() 参数**：
    - "emotion": 数据集名称。
    - split: 指定加载的拆分（可选，默认加载所有拆分）。
    - cache_dir: 自定义缓存路径,默认路径~/.cache/huggingface/datasets。
    - download_mode: 控制下载行为（"reuse_dataset_if_exists" 复用已下载数据）。
2. **map() 参数**：
    - function: 自定义处理函数（如分词）。
    - batched=True: 批量处理，提高效率。
    - remove_columns: 删除不需要的列（如原始 text）。
3. **filter() 参数**：
    - function: 过滤条件（返回布尔值）。
    - desc: 进度条描述。
4. **set_format() 参数**：
    - type: 输出格式（"torch", "tensorflow", "numpy", 或 None）。
    - columns: 指定转换的列。

**输出内容**

1. **原始数据集**：
    - 一个 Dataset 对象，包含特征和样本。
    - 访问方式类似字典或列表。
2. **预处理后数据集**：
    - 包含 tokenizer 输出（如 input_ids, attention_mask）。
    - 移除原始列后更适合模型输入。
3. **过滤后数据集**：
    - 仅保留满足条件的样本。
4. **PyTorch 格式**：
    - 数据转换为张量，直接可用于训练。

**批量访问**

- 切片（如 [:2]）返回一个字典，键对应列名，值是张量。
- 形状反映批量大小（batch_size）和序列长度。

**保存数据集**：python

```python
tokenized_dataset.save_to_disk("./my_dataset")
```

**动态填充**

将所有示例填充到该 batch 中最长元素的长度

```python
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

batch = data_collator(samples)
```

## Trainer API

准备数据示例

In [2]:
raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

定义`TrainingArguments` 类，包含超参数


In [ ]:
training_args = TrainingArguments(output_dir="hf_cache/test-trainer") 
#用于保存训练后的模型以及训练过程中的 checkpoint 的目录。
#对于其余的参数你可以保留默认值，这对于简单的微调应该效果就很好了。
from transformers import AutoModelForSequenceClassification
# Bert没有在句子分类的数据集上训练过，会报错，意思是已有的输出头没有使用，新的输出头是随机初始化的
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


定义一个 `Trainer` 把到目前为止构建的所有对象 —— `model` ，`training_args`，训练和验证数据集， `data_collator` 和 `tokenizer` 传递给 `Trainer`

In [ ]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [6]:
# 冻结BERT主干网络
for param in model.bert.parameters():
    param.requires_grad = False

# 只训练分类头
for param in model.classifier.parameters():
    param.requires_grad = True

In [7]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)
8
trainer.train()

/tmp/ipykernel_298799/2641408767.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
500,0.638200
1000,0.637900


TrainOutput(global_step=1377, training_loss=0.6373908791580145, metrics={'train_runtime': 627.4518, 'train_samples_per_second': 17.538, 'train_steps_per_second': 2.195, 'total_flos': 405114969714960.0, 'train_loss': 0.6373908791580145, 'epoch': 3.0})

**评估**

In [11]:
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)
# predict() 方法的输出一个带有三个字段的命名元组: predictions ,label_ids 和 metrics
# metrics 字段将只包含所传递的数据集的损失,以及一些时间指标(总共花费的时间和平均预测时间)
# predictions.prediction (batch,num_class)
preds = np.argmax(predictions.predictions, axis=-1)
metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

(408, 2) (408,)


{'accuracy': 0.6838235294117647, 'f1': 0.8122270742358079}

就比随机猜测高一点点。bert的语言潜能还没有完全适应新任务，只训练3个epoch太少了。微调还是用GPU吧。

最后把所有东西打包在一起，我们就得到了 `compute_metrics()` 函数

该函数必须接收一个 `EvalPrediction` 对象（它是一个带有 `predictions` 和 `label_ids` 字段的参数元组），并将返回一个字符串映射到浮点数的字典（字符串是返回的指标名称，而浮点数是其值）

In [12]:
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

为了查看模型在每个训练周期结束时的好坏，下面是我们如何使用 `compute_metrics()` 函数定义一个新的 `Trainer`

In [14]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/tmp/ipykernel_298799/1799240845.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


## Training Process